# Segmenting and Clustering Neighborhoods in Toronto PART 2

Scrape the wikipedia data found at - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
pull = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text,'lxml')

In [3]:
toronto_table = pull.find('table',{'class':'wikitable sortable'})
toronto_rows = toronto_table.find_all('tr')
neighborhoods = []

for row in toronto_rows:
    getlist = row.text.split('\n')[1:-1]
    
    neighborhoods.append(getlist)




##### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
import pandas as pd 

df_n = pd.DataFrame(neighborhoods[1:], columns = ['Postcode', 'Borough', 'Neighborhood'])




###### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df_nB = df_n[~df_n['Borough'].str.contains('Not assigned')]




###### More than one neighborhood can exist in one postal code area.These two rows will be combined into one row with the neighborhoods separated with a comma.

In [45]:
df_nN = df_nB.groupby(['Postcode','Borough'], sort=False).agg( ', '.join).reset_index()

###### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [49]:
df_nN.loc[df_nN['Neighborhood'] == 'Not assigned' , 'Neighborhood'] = df_nN['Borough']

In [8]:
df_nN.shape

(103, 3)

### Use link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [50]:
df_pull_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_pull_geo.rename({'Postal Code' : 'Postcode'},axis = 'columns', inplace = True)

### Use the csv file to create the dataframe.

In [51]:
df_nN = df_nB.groupby(['Postcode','Borough'], sort=False).agg( ', '.join).reset_index()
df_nN.loc[df_nN['Neighborhood'] == 'Not assigned' , 'Neighborhood'] = df_nN['Borough']
df_next = pd.merge(df_nN, df_pull_geo[['Postcode', 'Latitude', 'Longitude']], on = 'Postcode')
df_next.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
